# Projet du cours "Kernel methods" du master MVA

Il s'agit du code associé au kernel Linear, moins détaillé que les fichiers RBF et Polynomial qui présentent les résultats principaux.

## Chargement des modules

In [1]:
%pylab inline
import pandas
import cvxopt
import time
import numpy as np
import numpy.linalg as la
import scipy
from scipy.stats import mode
import matplotlib.pyplot as plt 

Populating the interactive namespace from numpy and matplotlib


# Importation des données

On importe les données du concours. On peut aussi insérer à la place de Xtr la base obtenue en appliquant les ondelettes pour voir les résultats obtenus avec les ondelettes.

In [2]:
df_train=pandas.read_csv('/Users/badr-eddinecherief-abdellatif/Documents/ENSAE/3A/MVA/Kernel methods/Projet ENS/Bases/Xtr.csv',header=None,sep=',')
df_train=df_train.drop(3072,1)
df_test=pandas.read_csv('/Users/badr-eddinecherief-abdellatif/Documents/ENSAE/3A/MVA/Kernel methods/Projet ENS/Bases/Xte.csv',header=None,sep=',')
df_test=df_test.drop(3072,1)
y_train=pandas.read_csv('/Users/badr-eddinecherief-abdellatif/Documents/ENSAE/3A/MVA/Kernel methods/Projet ENS/Bases/Ytr.csv',sep=';')
y_train=pandas.DataFrame(y_train['Prediction'])

# Répartition des labels
Nous travaillons sur un échantillon d'apprentissage dans le but de classifier des images. Vérifions que nous avons bien équirépartition des images dans l'échantillon d'apprentissage afin de pouvoir mener à bien notre modélisation

In [3]:
y_train["Prediction"].value_counts()

7    500
3    500
6    500
2    500
9    500
5    500
1    500
8    500
4    500
0    500
Name: Prediction, dtype: int64

# Implémentation des méthodes

Dans cette partie, nous implémenterons les différentes méthodes que nous testerons par la suite. Il s'agit des méthodes suivantes:
- SVM
- ACP
- HOG


## Support Vector Machine

In [4]:
class SVMTrainer(object):
    def __init__(self, c):
        self._c = c
        
    def train(self, X, y):
        lagrange_multipliers = self._compute_multipliers(X, y)
        return self._construct_predictor(X, y, lagrange_multipliers)

    def _gram_matrix(self, X):
        n_samples, n_features = X.shape
        K=dot(X.values,transpose(X.values))
        print("Taille de la matrice de Gram :",K.shape)
        return K

    def _construct_predictor(self, X, y, lagrange_multipliers):
        support_vector_indices=abs(lagrange_multipliers)>0
        support_multipliers = lagrange_multipliers[support_vector_indices] # alpha i positifs
        support_vectors = X[support_vector_indices] # vecteurs support
        support_vector_labels = y[support_vector_indices] # labels des vecteurs support
        print("Nombre de vecteurs supports :",len(support_vectors))
        pre=SVMPredictor(bias=0.0,weights=support_multipliers,
             support_vectors=support_vectors,support_vector_labels=support_vector_labels).predictest(support_vectors.values)[0]
        bias = np.mean(support_vector_labels[:,0]-pre)
        print("Biais:",bias)
        return SVMPredictor(bias=bias,weights=support_multipliers,support_vectors=support_vectors,
            support_vector_labels=support_vector_labels)

    def _compute_multipliers(self, X, y):
        n_samples, n_features = X.shape
        K=self._gram_matrix(X)
        P=cvxopt.matrix(K)
        q=cvxopt.matrix(-1*y[:,0])
        G_std=cvxopt.matrix(-1*np.diag(y[:,0]))
        h_std=cvxopt.matrix(np.zeros(n_samples))
        G_slack=cvxopt.matrix(np.diag(y[:,0]))
        h_slack=cvxopt.matrix(np.ones(n_samples) * self._c)
        G=cvxopt.matrix(np.vstack((G_std, G_slack)))
        h=cvxopt.matrix(np.vstack((h_std, h_slack)))
        A=cvxopt.matrix(np.ones(n_samples),(1,n_samples))
        b=cvxopt.matrix(0.0)
        solution = cvxopt.solvers.qp(P, q, G, h, A=None, b=None)
        return np.ravel(solution['x'])

In [5]:
class SVMPredictor(object):
    def __init__(self,bias,weights,support_vectors,support_vector_labels):
        self._bias = bias
        self._weights = weights
        self._support_vectors = support_vectors
        self._support_vector_labels = support_vector_labels

    def predictest(self,x):
        inter = dot(self._support_vectors,transpose(x))
        result = dot(transpose(self._weights),inter)+self._bias
        return sign(result),result

## Adaptation du SVM au cas multi-classe
On va tout d'abord implémenter la méthode One Vs One, i.e. considérer 10x9/2=45 problèmes de classifications binaires en traitant des paires d'instances, puis implémenter la méthode One Vs Rest, i.e. entraîner un classifieur par classe.

### One Vs One

In [6]:
def SVMOvO(data_train,data_test,is_train,C):
    SVM=SVMTrainer(C)
    result=(20*ones(len(data_test))).reshape(len(data_test),1)
    for i in [0,1,2,3,4,5,6,7,8]:
        for j in range(i+1,10):
            print("Classe :",i,j)
            df_train_ij=data_train[asmatrix(is_train,dtype=ndarray)==(i,j)].copy()
            is_train_ij=is_train[asmatrix(is_train,dtype=ndarray)==(i,j)].copy()
            is_train_ij[is_train==i]=1
            is_train_ij[is_train==j]=-1
            s1=time.clock()
            SVMPred=SVM.train(df_train_ij,asfarray(is_train_ij))
            s2=time.clock()
            print("Temps pour l'entrainement :",s2-s1)
            s3=time.clock()
            values=SVMPred.predictest(data_test.values)[0]
            s4=time.clock()
            print("Temps pour la prédiction :",s4-s3)
            values[values==1]=i
            values[values==-1]=j
            values=values.reshape(len(data_test),1)
            result=concatenate((result,values),1)
            is_test_predict=mode(result,axis=1)
    return(is_test_predict)    

### One Vs Rest

In [7]:
def SVMOvR(data_train,data_test,is_train,C):
    SVM=SVMTrainer(C)
    result=(-20*ones(len(data_test))).reshape(len(data_test),1)
    for i in [0,1,2,3,4,5,6,7,8,9]:
        print("Classe:",i)
        is_train_i=is_train.copy()
        is_train_i[is_train==i]=1
        is_train_i[is_train!=i]=-1
        s1=time.clock()
        SVMPred=SVM.train(data_train,asfarray(is_train_i))
        s2=time.clock()
        print("Temps pour l'entrainement:",s2-s1)
        s3=time.clock()
        values=SVMPred.predictest(data_test.values)[1]
        s4=time.clock()
        print("Temps pour la prédiction:",s4-s3)
        values=values.reshape(len(data_test),1)
        result=concatenate((result,values),1)
    predict=argmax(result,axis=1)-1
    return(predict)  

# Analyse en Composantes Principales (non-kernel)
Il s'agit d'une ACP toute simple, où on détermine les valeurs propres et vecteurs propres correspondant à la matrice de variance de la base de données.

In [8]:
class ACP(object):
    def __init__(self, n_components):
        self.n_components = n_components
         
    def center(self,X):
        result = X - mean(X,axis=0)
        return result

    def vpropres(self,X):
        n_samples,n_var = X.shape
        X_centre = self.center(X)
        Var = (1/n_samples)*dot(X_centre.T,X_centre)
        de = diag(var(Var,axis=1))
        cool = dot(Var,transpose(de))
        vp = scipy.linalg.eig(cool)
        a = sorted(vp[0],reverse=True)
        stop = a[self.n_components-1]
        vect = vp[1][:,vp[0]>=stop]
        print("Variance expliquée :" , real(sum(vp[0][vp[0]>=stop]))/real(sum(vp[0])))
        return(vect)
    
    def fit_transform(self,X):
        X_centre = self.center(X)
        vect = self.vpropres(X_centre)
        X_transform = dot(X,vect)
        return(pandas.DataFrame(real(X_transform)))

# Tests
Afin de tester nos modèles sur nos données d'entraînement, on va découper notre base d'entraînement en train/test à hauteur de 60/40%, et effectuer les tests avec chaque méthode sans HOG. Nous avons constaté après tests que les résultats sont plus mauvais qu'avec les noyaux RBF et Polynomial, et nous n'approfondirons donc pas ce noyau et nous nous centrerons donc sur les autres. Nous présentons les tests sans HOG et avec la méthode SVM One-Vs-One.

In [9]:
X_train_1 = df_train[0:3000]
X_test_1 = df_train[3000:5000]
Y_train_1 = y_train[0:3000]
Y_test_1 = y_train[3000:5000]
print(X_train_1.shape)
print(X_test_1.shape)
Y_train_1["Prediction"].value_counts()

(3000, 3072)
(2000, 3072)


3    311
1    308
0    306
6    305
8    304
4    298
5    294
7    292
9    291
2    291
Name: Prediction, dtype: int64

## Test SVM One Vs One

In [15]:
y_predict_OvO=SVMOvO(X_train_1,X_test_1,Y_train_1,6)

Classe : 0 1
Taille de la matrice de Gram : (614, 614)
     pcost       dcost       gap    pres   dres
 0:  5.1524e+02 -1.6610e+04  4e+04  7e-01  5e-15
 1:  6.2493e+02 -4.9783e+03  6e+03  3e-03  5e-15
 2:  1.8132e+01 -8.7615e+02  9e+02  3e-04  4e-15
 3: -1.2061e+02 -2.2191e+02  1e+02  7e-06  3e-15
 4: -1.3451e+02 -1.5135e+02  2e+01  9e-07  2e-15
 5: -1.3718e+02 -1.4004e+02  3e+00  1e-07  2e-15
 6: -1.3775e+02 -1.3796e+02  2e-01  6e-09  2e-15
 7: -1.3781e+02 -1.3781e+02  7e-03  1e-10  2e-15
 8: -1.3781e+02 -1.3781e+02  3e-04  3e-12  2e-15
 9: -1.3781e+02 -1.3781e+02  1e-05  3e-14  2e-15
Optimal solution found.
Nombre de vecteurs supports : 614
Biais: 0.0
Temps pour l'entrainement : 0.6944069999999982
Temps pour la prédiction : 0.5414600000000007
Classe : 0 2
Taille de la matrice de Gram : (597, 597)
     pcost       dcost       gap    pres   dres
 0:  4.9456e+02 -1.3794e+04  2e+04  4e-01  5e-15
 1:  2.7308e+02 -2.1553e+03  2e+03  2e-16  5e-15
 2: -1.2558e+02 -5.3801e+02  4e+02  2e-16  4

In [16]:
compt=Y_test_1.values-y_predict_OvO.mode
print("Pourcentage de bonnes prédictions :",sum(compt==0)/len(Y_test_1))

Pourcentage de bonnes prédictions : 0.1715


## Test SVM One Vs One après ACP

### On applique une ACP et on conserve 30 composantes principales 

In [10]:
acp=ACP(30)
X_train_acp=acp.fit_transform(df_train)
print(X_train_acp.shape)

Variance expliquée : 0.220212319462
(5000, 30)


In [14]:
X_train_2 = X_train_acp[0:3000]
X_test_2 = X_train_acp[3000:5000]

In [15]:
y_predict_OvO_ACP=SVMOvO(X_train_2,X_test_2,Y_train_1,6)

Classe : 0 1
Taille de la matrice de Gram : (614, 614)
     pcost       dcost       gap    pres   dres
 0: -1.6419e+03 -2.1227e+04  4e+04  7e-01  6e-15
 1: -1.6481e+03 -6.6264e+03  5e+03  2e-16  7e-15
 2: -2.1147e+03 -3.1026e+03  1e+03  2e-16  7e-15
 3: -2.4645e+03 -2.7867e+03  3e+02  2e-16  8e-15
 4: -2.5544e+03 -2.6854e+03  1e+02  2e-16  8e-15
 5: -2.5942e+03 -2.6369e+03  4e+01  2e-16  9e-15
 6: -2.6069e+03 -2.6224e+03  2e+01  2e-16  9e-15
 7: -2.6124e+03 -2.6159e+03  3e+00  2e-16  1e-14
 8: -2.6136e+03 -2.6147e+03  1e+00  2e-16  9e-15
 9: -2.6139e+03 -2.6142e+03  3e-01  2e-16  9e-15
10: -2.6141e+03 -2.6141e+03  6e-02  2e-16  1e-14
11: -2.6141e+03 -2.6141e+03  7e-03  2e-16  1e-14
12: -2.6141e+03 -2.6141e+03  8e-05  2e-16  1e-14
Optimal solution found.
Nombre de vecteurs supports : 614
Biais: 0.153094462541
Temps pour l'entrainement : 0.8634470000000078
Temps pour la prédiction : 0.018660999999980277
Classe : 0 2
Taille de la matrice de Gram : (597, 597)
     pcost       dcost       g

In [16]:
compt=Y_test_1.values-y_predict_OvO_ACP.mode
print("Pourcentage de bonnes prédictions:",sum(compt==0)/len(Y_test_1))

Pourcentage de bonnes prédictions: 0.1705


## Optimisation des paramètres pour le SVM One Vs One

In [17]:
y_predict_OvO = [SVMOvO(X_train_1,X_test_1,Y_train_1,6)]*10
for C in range(10) :
    y_predict_OvO[C] = SVMOvO(X_train_1,X_test_1,Y_train_1,C)

Classe : 0 1
Taille de la matrice de Gram : (614, 614)
     pcost       dcost       gap    pres   dres
 0:  5.1524e+02 -1.6610e+04  4e+04  7e-01  5e-15
 1:  6.2493e+02 -4.9783e+03  6e+03  3e-03  5e-15
 2:  1.8132e+01 -8.7615e+02  9e+02  3e-04  4e-15
 3: -1.2061e+02 -2.2191e+02  1e+02  7e-06  3e-15
 4: -1.3451e+02 -1.5135e+02  2e+01  9e-07  2e-15
 5: -1.3718e+02 -1.4004e+02  3e+00  1e-07  2e-15
 6: -1.3775e+02 -1.3796e+02  2e-01  6e-09  2e-15
 7: -1.3781e+02 -1.3781e+02  7e-03  1e-10  2e-15
 8: -1.3781e+02 -1.3781e+02  3e-04  3e-12  2e-15
 9: -1.3781e+02 -1.3781e+02  1e-05  3e-14  2e-15
Optimal solution found.
Nombre de vecteurs supports : 614
Biais: 0.0
Temps pour l'entrainement : 0.7530319999999904
Temps pour la prédiction : 0.25462799999999675
Classe : 0 2
Taille de la matrice de Gram : (597, 597)
     pcost       dcost       gap    pres   dres
 0:  4.9456e+02 -1.3794e+04  2e+04  4e-01  5e-15
 1:  2.7308e+02 -2.1553e+03  2e+03  2e-16  5e-15
 2: -1.2558e+02 -5.3801e+02  4e+02  2e-16  

In [18]:
for C in range(10) :
    compt=Y_test_1.values-y_predict_OvO[C].mode
    print( C , sum(compt==0)/len(Y_test_1))

0 0.0945
1 0.1765
2 0.171
3 0.1725
4 0.1715
5 0.1715
6 0.1715
7 0.1715
8 0.1715
9 0.1715


In [19]:
## Paramètre optimal : pour C plus grand que 0 (sinon ce paramètre n'a pas de sens), la valeur optimale en SVM 
## One-Vs-One est C=1 
C = 1

# Comparaison avec le package Scikit-Learn

In [21]:
from sklearn.svm import SVC
clf = SVC(kernel="rbf",C=6,gamma=1/(2*2**2),degree=2)
clf.fit(X_train_1,Y_train_1)
compt = Y_test_1.values[:,0] - clf.predict(X_test_1)
print("Pourcentage de bonnes prédictions :",sum(compt==0)/len(Y_test_1))

//anaconda/lib/python3.5/site-packages/sklearn/svm/base.py:514: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y_ = column_or_1d(y, warn=True)


Pourcentage de bonnes prédictions : 0.22


# Les résultats sont moins bons que pour les kernels RBF et Polynomial. 

# Nous poursuivrons l'étude avec les noyaux RBF et Polynomial.